In [2]:
# ==============================
# 1. Setup and Installation
# ==============================

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Check CUDA and driver versions
!nvcc --version  # Check CUDA version
!nvidia-smi      # Check driver version

# Install RAPIDS and other required libraries
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Tue Oct 28 18:34:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+===========

In [3]:
## After restarting, install remaining necessary libraries
# Run this cell after restarting the runtime
!pip install bertopic==0.16.3
!pip install octis
!pip install sentence-transformers
!pip install umap-learn==0.5.3  # Specify a compatible version
!pip install hdbscan
!pip install tqdm
!pip install pandas
!pip install gensim
!pip install wandb
!pip install umap
!pip install scipy
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82810 sha256=46c45b87b553d3c930957fe4d9e3e745bb303dfe13cea6f095bd7cb78d451712
  St

In [ ]:
!pip list | grep umap

umap                                  0.1.1
umap-learn                            0.5.3


In [ ]:
!pip uninstall -y umap

Found existing installation: umap 0.1.1
Uninstalling umap-0.1.1:
  Successfully uninstalled umap-0.1.1


In [ ]:
!find . -type d -name "__pycache__" -exec rm -r {} +

In [ ]:
!pip install --upgrade bertopic umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: umap-learn
    Found existing installation: umap-learn 0.5.3
    Uninstalling umap-learn-0.5.3:
      Successfully uninstalled umap-learn-0.5.3
  Attempting uninstall: bertopic
    Found existing installation: bertopic 0.16.3
    Uninstalling bertopic-0.16.3:
      Successfully uninstalled bertopic-0.16.3


In [ ]:
import os
import time
import logging
import traceback
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from bertopic import BERTopic
from tenacity import (
    retry,
    stop_after_attempt,
    wait_exponential,
    retry_if_exception_type
)
import requests
import json

# ========== CONFIGURATION ==========
OPENROUTER_API_KEY = "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"
os.environ["OPENAI_API_KEY"] = OPENROUTER_API_KEY

# Primary and fallback models
PRIMARY_MODEL = "perplexity/sonar-deep-research"
FALLBACK_MODEL = "openai/gpt-3.5-turbo"

# File paths
FAILED_LOG_PATH = "topic_labeling_errors.log"
MODEL_PATH = "/content/drive/MyDrive/BERTTopic_Models_POS/paraphrase-mpnet-base-v2_Model_1_POS_a/bertopic_model_iter_1_paraphrase-mpnet-base-v2_POS_a.pkl"
OUTPUT_CSV = "/content/drive/MyDrive/BERTTopic_Models_POS/verified_labeled_topics.csv"
TOPIC_LIMIT = 364
DEBUG_MODE = True  # Set to True for verbose logging

# ========== ENHANCED PROMPT TEMPLATE ==========
PROMPT_TEMPLATE = """
Generate a topic label with EXACTLY 6 TAB-SEPARATED components based on these detailed guidelines:

1. CHARACTER DYNAMICS: Relationships, interactions, or conflicts between characters
   • Example types: "Rivalry Turned Alliance", "Unrequited Love", "Mentor-Protégé Conflict"
   • If keywords like "friend", "love", "trust", "betray" appear, include this component
   • Use "None" if no clear character dynamics are present

2. THEMES: Abstract ideas representing moral, philosophical, or ideological concerns
   • Example types: "Duty vs. Desire", "The Power of Redemption", "Class Struggles"
   • ALWAYS include a theme, even if abstract or general
   • For objects/places, focus on what they represent (e.g., "doors" → "Transitions and Boundaries")

3. SETTINGS: Physical locations or conceptual environments
   • Example types: "Haunted Castle", "War-Torn Battlefield", "Urban Underworld"
   • Use "None" if no clear setting is indicated in keywords
   • Be specific but not overly speculative

4. ACTIONS & EVENTS: Plot-driven occurrences advancing the narrative
   • Example types: "Betrayal and Retribution", "Quest for Identity", "Escape from Captivity"
   • Focus on actions mentioned in keywords
   • Use "None" if no clear events are indicated

5. EMOTIONAL ATMOSPHERE: Overall mood and emotional impact
   • Format: "Mood - [emotion]" (e.g., "Mood - Cheerful", "Mood - Tense")
   • Use "Mood - Cheerful" for positive emotions (smile, laugh)
   • Use "Mood - Tense" for conflict/danger (war, fight)
   • Default to "Mood - Neutral" for neutral topics

6. REPRESENTATION: How the topic is portrayed
   • Format: "Portrayal - [tone]" (e.g., "Portrayal - Positive", "Portrayal - Serious")
   • Use "Portrayal - Positive" for topics portrayed favorably
   • Use "Portrayal - Serious" for grave or solemn topics
   • Default to "Portrayal - Neutral" for neutral topics

IMPORTANT EXAMPLES FROM EXISTING TOPICS:

Topic: smile, laugh, grin
Friendly Camaraderie\tJoy and Lightheartedness\tJoyous Social Gatherings\tFestive or Cheerful Gatherings\tMood - Cheerful\tPortrayal - Positive

Topic: war, combat, battle
Allies and Enemies in Battle\tConflict and Warfare\tWar-Torn Battlefield\tWar and Combat\tMood - Tense\tPortrayal - Serious

Topic: door, doorway, entrance
None\tTransitions and Boundaries\tEntrances & Corridors\tEntering and Exiting\tMood - Neutral\tPortrayal - Neutral

Topic: kiss, kissed, embrace
Romantic Intimacy\tLove and Affection\tPrivate Romantic Spaces\tKissing and Embracing\tMood - Affectionate\tPortrayal - Positive

RULES:
- Use exactly ONE tab (\\t) between components
- Be descriptive but concise for each component
- Use "None" when appropriate for dynamics, settings, or events
- ALWAYS provide a theme (never "None" for theme component)
- Follow the mood and portrayal formats exactly
- Return ONLY the 6 tab-separated components with no other text

YOUR TASK:
For the following keywords, provide 6 tab-separated components following the guidelines above.

TOPIC KEYWORDS: [KEYWORDS]
REPRESENTATIVE DOCUMENTS:
[DOCUMENTS]
"""

# ========== LABEL GENERATOR USING OPENROUTER ==========
class TopicLabelGenerator:
    def __init__(self):
        self.api_key = os.environ["OPENAI_API_KEY"]
        self.primary_model = PRIMARY_MODEL
        self.fallback_model = FALLBACK_MODEL
        self.prompt = PROMPT_TEMPLATE
        self.api_url = "https://openrouter.ai/api/v1/chat/completions"
        self.debug = DEBUG_MODE

    def generate_label(self, keywords, documents):
        """Main method to generate labels with fallback mechanism"""
        try:
            if self.debug:
                print(f"\nTrying primary model: {self.primary_model}")

            return self._call_api(self.primary_model, keywords, documents)

        except Exception as e:
            if self.debug:
                print(f"Primary model failed: {str(e)[:200]}...")
                print(f"Trying fallback model: {self.fallback_model}")

            try:
                return self._call_api(self.fallback_model, keywords, documents)

            except Exception as e2:
                if self.debug:
                    print(f"Fallback model also failed: {str(e2)[:200]}...")
                    print("Using default label generator")

                return self._create_fallback_label(keywords)

    @retry(
        stop=stop_after_attempt(3),
        wait=wait_exponential(multiplier=1, min=2, max=30),
        retry=retry_if_exception_type((requests.exceptions.RequestException, ValueError)),
        reraise=True
    )
    def _call_api(self, model, keywords, documents):
        """Make the actual API call with retries"""
        truncated_docs = [doc[:300] + "..." if len(doc) > 300 else doc for doc in documents[:3]]
        prompt = self.prompt.replace("[KEYWORDS]", ", ".join(keywords))
        prompt = prompt.replace("[DOCUMENTS]", "\n---\n".join(truncated_docs))

        payload = {
            "model": model,
            "messages": [{"role": "user", "content": prompt}]
        }

        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://yourdomain.com",
            "X-Title": "BERTopic_Labeling_Pipeline"
        }

        try:
            response = requests.post(self.api_url, headers=headers, json=payload, timeout=60)

            # Debug response info
            if self.debug:
                print(f"Status code: {response.status_code}")

            if response.status_code != 200:
                error_msg = f"HTTP {response.status_code}: {response.text}"
                if self.debug:
                    print(f"API error: {error_msg}")
                raise ValueError(error_msg)

            try:
                result = response.json()
            except json.JSONDecodeError:
                if self.debug:
                    print(f"Failed to parse JSON from response: {response.text[:500]}...")
                raise ValueError("Invalid JSON response")

            # Verbose debugging of the response
            if self.debug:
                print(f"Response keys: {list(result.keys())}")

            # Check if choices exists
            if 'choices' not in result:
                if self.debug:
                    print(f"Missing 'choices' key. Response: {json.dumps(result)[:500]}...")
                raise ValueError(f"Missing 'choices' in response: {json.dumps(result)[:200]}...")

            if not result['choices'] or not isinstance(result['choices'], list):
                if self.debug:
                    print(f"Invalid 'choices' format: {result.get('choices', 'None')}")
                raise ValueError("Invalid choices format in response")

            try:
                content = result["choices"][0]["message"]["content"].strip()
                return self._validate_label(content, keywords)
            except (KeyError, IndexError, TypeError) as e:
                if self.debug:
                    print(f"Error extracting content: {str(e)}")
                    print(f"Choices structure: {json.dumps(result.get('choices', []))[:300]}...")
                raise ValueError(f"Bad response format: {str(e)}")

        except requests.exceptions.RequestException as e:
            if self.debug:
                print(f"Request exception: {str(e)}")
            raise

    def _create_fallback_label(self, keywords):
        """Create a fallback label when both APIs fail"""
        topic_name = keywords[0].capitalize() if keywords else "Unknown Topic"
        secondary_word = keywords[1].capitalize() if len(keywords) > 1 else ""
        theme = f"{topic_name} and {secondary_word}" if secondary_word else f"{topic_name} and Related Concepts"

        return {
            "Character_Dynamics": "None",
            "Themes": theme,
            "Settings": "None",
            "Actions_Events": "None",
            "Emotional_Atmosphere": "Mood - Neutral",
            "Representation": "Portrayal - Neutral"
        }

    def _validate_label(self, label, keywords):
        """Validate and fix the label structure if needed"""
        if not label or not isinstance(label, str):
            if self.debug:
                print(f"Invalid label response: {label}")
            return self._create_fallback_label(keywords)

        try:
            parts = label.split('\t')

            # Handle case where model doesn't use tabs properly
            if len(parts) == 1:
                parts = label.split('   ')

            # If still not good, try line breaks
            if len(parts) == 1:
                parts = label.split('\n')

            # Filter out empty parts
            parts = [p for p in parts if p.strip()]

            if self.debug:
                print(f"Split result: {parts} (length: {len(parts)})")

            # If we don't have 6 components, create reasonable defaults
            if len(parts) != 6:
                if self.debug:
                    print(f"Expected 6 components, got {len(parts)}: {parts}")
                return self._create_fallback_label(keywords)

            dynamic, theme, setting, action_event, mood, portrayal = [p.strip() for p in parts]

            # Fix common issues
            if not theme or theme.lower() == "none":
                theme = f"{keywords[0].capitalize()} and Related Concepts" if keywords else "Topic"
            if not mood.startswith("Mood - "):
                mood = "Mood - " + mood
            if not portrayal.startswith("Portrayal - "):
                portrayal = "Portrayal - " + portrayal

            return {
                "Character_Dynamics": dynamic,
                "Themes": theme,
                "Settings": setting,
                "Actions_Events": action_event,
                "Emotional_Atmosphere": mood,
                "Representation": portrayal
            }

        except Exception as e:
            if self.debug:
                print(f"Label validation error: {str(e)}")
            return self._create_fallback_label(keywords)

# ========== PIPELINE FUNCTIONS ==========

def initialize_logging():
    """Set up logging to file and console"""
    logging.basicConfig(
        filename=FAILED_LOG_PATH,
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - Topic %(topic_id)s: %(message)s",
        filemode='w'
    )
    console = logging.StreamHandler()
    console.setLevel(logging.INFO)
    formatter = logging.Formatter('%(message)s')
    console.setFormatter(formatter)
    logging.getLogger('').addHandler(console)

def load_model():
    """Load the BERTopic model from disk"""
    print(f"\n🔍 Loading BERTopic model from: {MODEL_PATH}")
    try:
        model = BERTopic.load(MODEL_PATH)
        topic_info = model.get_topic_info()
        print(f"✅ Successfully loaded model with {len(topic_info)} topics")
        print("\nSample topics:")
        print(topic_info.head(3))
        return model
    except Exception as e:
        logging.critical(f"Model loading failed: {str(e)}")
        raise RuntimeError(f"Could not load model: {str(e)}")

def process_topics(model, limit=TOPIC_LIMIT):
    """Process topics to generate labels"""
    generator = TopicLabelGenerator()
    results = []

    # Get topics (excluding -1 which is typically outliers/noise)
    valid_topics = [t for t in model.get_topics() if t != -1][:limit]
    print(f"Processing {len(valid_topics)} topics...")

    for topic_id in tqdm(valid_topics, desc="Labeling Topics"):
        try:
            # Extract keywords and documents
            keywords = [w for w, _ in model.get_topic(topic_id)[:8]]
            docs = model.get_representative_docs(topic_id) or ["No documents available"]

            # Try to generate the label
            label_data = generator.generate_label(keywords, docs)

            # Store the result
            result = {
                "Topic_ID": topic_id,
                "Keywords": ", ".join(keywords),
                **label_data,
                "Sample_Document": docs[0][:200] + "..." if docs else "No document",
                "Status": "Success"
            }
            results.append(result)

            # Small delay to avoid rate limits
            time.sleep(1)

        except Exception as e:
            error_msg = str(e)[:200]
            logging.error(
                f"Topic {topic_id} processing failed: {error_msg}",
                extra={"topic_id": topic_id}
            )

            # Still add an entry with the error info
            fallback_theme = f"{keywords[0].capitalize()} and Related Concepts" if 'keywords' in locals() and keywords else "Unknown"
            results.append({
                "Topic_ID": topic_id,
                "Keywords": ", ".join(keywords) if 'keywords' in locals() else "Unknown",
                "Character_Dynamics": "Error",
                "Themes": fallback_theme,
                "Settings": "Error",
                "Actions_Events": "Error",
                "Emotional_Atmosphere": "Mood - Neutral",
                "Representation": "Portrayal - Neutral",
                "Sample_Document": docs[0][:200] + "..." if 'docs' in locals() and docs else "",
                "Status": f"Error: {error_msg}"
            })

    return pd.DataFrame(results)

def save_results(df):
    """Save results to CSV with timestamp"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = os.path.dirname(OUTPUT_CSV)

    # Create directory if it doesn't exist
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    output_path = f"{os.path.splitext(OUTPUT_CSV)[0]}_{timestamp}.csv"

    # Calculate success rate
    success_rate = len(df[df['Status'] == 'Success']) / len(df) if len(df) > 0 else 0
    df['Metadata'] = f"Success rate: {success_rate:.1%} | Generated: {timestamp}"

    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"\n💾 Saved results to: {output_path}")
    print(f"✅ Success rate: {success_rate:.1%}")

    # Print sample successful labels
    if success_rate > 0:
        print("\nSample successful labels:")
        success_df = df[df['Status'] == 'Success'].head(3)
        for _, row in success_df.iterrows():
            print(f"\nTopic {row['Topic_ID']}: {row['Keywords']}")
            print(f"  • Character Dynamics: {row['Character_Dynamics']}")
            print(f"  • Theme: {row['Themes']}")
            print(f"  • Setting: {row['Settings']}")
            print(f"  • Actions/Events: {row['Actions_Events']}")
            print(f"  • Mood: {row['Emotional_Atmosphere']}")
            print(f"  • Portrayal: {row['Representation']}")

    return output_path

def main():
    """Main pipeline function"""
    initialize_logging()
    try:
        model = load_model()
        print(f"\n🚀 Starting to process up to {TOPIC_LIMIT} topics...")
        start_time = time.time()
        results_df = process_topics(model)
        output_file = save_results(results_df)
        elapsed = time.time() - start_time
        print(f"\n🏁 Process completed in {elapsed:.1f} seconds")
        print(f"📊 Results saved to: {output_file}")
        print(f"📋 Error log: {FAILED_LOG_PATH}")
    except Exception as e:
        logging.critical(f"Pipeline failed: {str(e)}\n{traceback.format_exc()}")
        print(f"⛔ Critical error: {str(e)}")
        print(f"Stack trace:\n{traceback.format_exc()}")
        exit(1)

if __name__ == "__main__":
    main()



🔍 Loading BERTopic model from: /content/drive/MyDrive/BERTTopic_Models_POS/paraphrase-mpnet-base-v2_Model_1_POS_a/bertopic_model_iter_1_paraphrase-mpnet-base-v2_POS_a.pkl
✅ Successfully loaded model with 364 topics

Sample topics:
   Topic   Count                             Name  \
0     -1  113001  -1_thought_think_thoughts_speak   
1      0    4846   0_smiled_smile_smiles_grinning   
2      1    2847     1_dress_dresses_gowns_attire   

                                      Representation  \
0  [thought, think, thoughts, speak, seemed, say,...   
1  [smiled, smile, smiles, grinning, smirking, gr...   
2  [dress, dresses, gowns, attire, clothing, gown...   

                                               NOUNS  \
0  [hands, way, words, arms, room, body, arm, mom...   
1  [smile, grin, smiles, joke, laughs, chuckle, g...   
2  [dress, hat, shirt, gown, trousers, clothes, j...   

                                 Representative_Docs  
0  [says hard say words inside released hit chest.

Labeling Topics:   0%|          | 0/363 [00:00<?, ?it/s]


Trying primary model: perplexity/sonar-deep-research
Status code: 200
Response keys: ['id', 'provider', 'model', 'object', 'created', 'choices', 'citations', 'usage']
Split result: ['Friendly Camaraderie', 'Joy and Lightheartedness', 'Joyous Social Gatherings', 'Festive or Cheerful Gatherings', 'Mood - Cheerful', 'Portrayal - Positive'] (length: 6)


Labeling Topics:   0%|          | 1/363 [00:43<4:20:52, 43.24s/it]


Trying primary model: perplexity/sonar-deep-research
Status code: 200
Response keys: ['id', 'provider', 'model', 'object', 'created', 'choices', 'citations', 'usage']
Split result: ['None', 'Clothing as Identity and Social Stratification', 'Social and Cultural Spheres', 'Dressing and Adornment', 'Mood - Contemplative', 'Portrayal - Serious'] (length: 6)


...


Trying primary model: perplexity/sonar-deep-research
Status code: 200
Response keys: ['id', 'provider', 'model', 'object', 'created', 'choices', 'citations', 'usage']
Split result: ['None', 'Physical Foundations and Symbolism', 'None', 'Mobility and Anatomical Movements', 'Mood - Neutral', 'Portrayal - Neutral'] (length: 6)


Labeling Topics:  17%|█▋        | 60/363 [47:33<4:01:24, 47.80s/it]


Trying primary model: perplexity/sonar-deep-research
Status code: 200
Response keys: ['id', 'provider', 'model', 'object', 'created', 'choices', 'citations', 'usage']
Split result: ['None', 'Passage of Time and Deadlines', 'None', 'Time Progression and Deadlines', 'Mood - Tense', 'Portrayal - Serious'] (length: 6)


Labeling Topics:  17%|█▋        | 61/363 [48:10<3:45:06, 44.73s/it]


Trying primary model: perplexity/sonar-deep-research


In [ ]:
import os
import time
import logging
import traceback
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from bertopic import BERTopic
from tenacity import (
    retry,
    stop_after_attempt,
    wait_exponential,
    retry_if_exception_type
)
import openai

# ========== CONFIGURATION ==========
OPENROUTER_API_KEY = "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"  # Replace with your actual key
os.environ["OPENAI_API_KEY"] = OPENROUTER_API_KEY

MODEL_NAME = "deepseek/deepseek-r1:free"
FAILED_LOG_PATH = "topic_labeling_errors.log"
MODEL_PATH = "/content/drive/MyDrive/BERTTopic_Models_POS/paraphrase-mpnet-base-v2_Model_1_POS_a/bertopic_model_iter_1_paraphrase-mpnet-base-v2_POS_a.pkl"
OUTPUT_CSV = "/content/drive/MyDrive/BERTTopic_Models_POS/verified_labeled_topics.csv"
TOPIC_LIMIT = 10  # Start small, then increase

# ========== ENHANCED PROMPT TEMPLATE WITH YOUR DETAILED GUIDELINES ==========
PROMPT_TEMPLATE = """
Generate a topic label with EXACTLY 6 TAB-SEPARATED components based on these detailed guidelines:

1. CHARACTER DYNAMICS: Relationships, interactions, or conflicts between characters
   • Example types: "Rivalry Turned Alliance", "Unrequited Love", "Mentor-Protégé Conflict"
   • If keywords like "friend", "love", "trust", "betray" appear, include this component
   • Use "None" if no clear character dynamics are present

2. THEMES: Abstract ideas representing moral, philosophical, or ideological concerns
   • Example types: "Duty vs. Desire", "The Power of Redemption", "Class Struggles"
   • ALWAYS include a theme, even if abstract or general
   • For objects/places, focus on what they represent (e.g., "doors" → "Transitions and Boundaries")

3. SETTINGS: Physical locations or conceptual environments
   • Example types: "Haunted Castle", "War-Torn Battlefield", "Urban Underworld"
   • Use "None" if no clear setting is indicated in keywords
   • Be specific but not overly speculative

4. ACTIONS & EVENTS: Plot-driven occurrences advancing the narrative
   • Example types: "Betrayal and Retribution", "Quest for Identity", "Escape from Captivity"
   • Focus on actions mentioned in keywords
   • Use "None" if no clear events are indicated

5. EMOTIONAL ATMOSPHERE: Overall mood and emotional impact
   • Format: "Mood - [emotion]" (e.g., "Mood - Cheerful", "Mood - Tense")
   • Use "Mood - Cheerful" for positive emotions (smile, laugh)
   • Use "Mood - Tense" for conflict/danger (war, fight)
   • Default to "Mood - Neutral" for neutral topics

6. REPRESENTATION: How the topic is portrayed
   • Format: "Portrayal - [tone]" (e.g., "Portrayal - Positive", "Portrayal - Serious")
   • Use "Portrayal - Positive" for topics portrayed favorably
   • Use "Portrayal - Serious" for grave or solemn topics
   • Default to "Portrayal - Neutral" for neutral topics

IMPORTANT EXAMPLES FROM EXISTING TOPICS:

Topic: smile, laugh, grin
Friendly Camaraderie\tJoy and Lightheartedness\tJoyous Social Gatherings\tFestive or Cheerful Gatherings\tMood - Cheerful\tPortrayal - Positive

Topic: war, combat, battle
Allies and Enemies in Battle\tConflict and Warfare\tWar-Torn Battlefield\tWar and Combat\tMood - Tense\tPortrayal - Serious

Topic: door, doorway, entrance
None\tTransitions and Boundaries\tEntrances & Corridors\tEntering and Exiting\tMood - Neutral\tPortrayal - Neutral

Topic: kiss, kissed, embrace
Romantic Intimacy\tLove and Affection\tPrivate Romantic Spaces\tKissing and Embracing\tMood - Affectionate\tPortrayal - Positive

RULES:
- Use exactly ONE tab (\\t) between components
- Be descriptive but concise for each component
- Use "None" when appropriate for dynamics, settings, or events
- ALWAYS provide a theme (never "None" for theme component)
- Follow the mood and portrayal formats exactly
- Return ONLY the 6 tab-separated components with no other text

YOUR TASK:
For the following keywords, provide 6 tab-separated components following the guidelines above.

TOPIC KEYWORDS: [KEYWORDS]
REPRESENTATIVE DOCUMENTS:
[DOCUMENTS]
"""

# ========== ROBUST LABEL GENERATOR ==========
class TopicLabelGenerator:
    def __init__(self):
        self.client = openai.OpenAI(
            api_key=os.environ["OPENAI_API_KEY"],
            base_url="https://openrouter.ai/api/v1",
            timeout=30  # Timeout set at client level
        )
        self.model = MODEL_NAME
        self.prompt = PROMPT_TEMPLATE

    @retry(
        stop=stop_after_attempt(5),
        wait=wait_exponential(multiplier=1, min=4, max=20),
        retry=retry_if_exception_type(
            (openai.APITimeoutError,
             openai.APIError,
             openai.APIConnectionError)
        ),
        reraise=True
    )
    def generate_label(self, keywords, documents):
        """Generate and validate topic label with retry logic"""
        try:
            # Prepare prompt with safe document truncation
            truncated_docs = [doc[:500] + "..." if len(doc) > 500 else doc
                            for doc in documents[:3]]

            prompt = self.prompt.replace("[KEYWORDS]", ", ".join(keywords))
            prompt = prompt.replace("[DOCUMENTS]", "\n---\n".join(truncated_docs))

            # API call without request_timeout parameter
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3,
                max_tokens=250
            )

            if not response.choices:
                raise ValueError("Empty API response")

            content = response.choices[0].message.content.strip()
            return self._validate_label(content, keywords)

        except Exception as e:
            logging.error(f"Label generation failed: {str(e)}")
            raise

    def _validate_label(self, label, keywords):
        """Strict validation of label format"""
        if not label:
            raise ValueError("Empty label response")

        parts = label.split('\t')
        if len(parts) != 6:
            raise ValueError(f"Expected 6 components, got {len(parts)}")

        # Make sure we have valid parts
        dynamic = parts[0].strip()
        theme = parts[1].strip()
        setting = parts[2].strip()
        action_event = parts[3].strip()
        mood = parts[4].strip()
        portrayal = parts[5].strip()

        # Ensure theme is never None
        if not theme or theme.lower() == "none":
            theme = f"{keywords[0].capitalize()} and Related Concepts" if keywords else "Topic"

        # Ensure mood has proper format
        if not mood.startswith("Mood - "):
            mood = "Mood - " + mood

        # Ensure portrayal has proper format
        if not portrayal.startswith("Portrayal - "):
            portrayal = "Portrayal - " + portrayal

        # Create result dictionary
        result = {
            "Character_Dynamics": dynamic,
            "Themes": theme,
            "Settings": setting,
            "Actions_Events": action_event,
            "Emotional_Atmosphere": mood,
            "Representation": portrayal
        }

        return result

# ========== PROCESSING PIPELINE ==========
def initialize_logging():
    """Configure logging with timestamp and topic context"""
    logging.basicConfig(
        filename=FAILED_LOG_PATH,
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - Topic %(topic_id)s: %(message)s",
        filemode='w'
    )
    console = logging.StreamHandler()
    console.setLevel(logging.INFO)
    formatter = logging.Formatter('%(message)s')
    console.setFormatter(formatter)
    logging.getLogger('').addHandler(console)

def load_model():
    """Load BERTopic model with validation"""
    print(f"\n🔍 Loading BERTopic model from: {MODEL_PATH}")
    try:
        model = BERTopic.load(MODEL_PATH)
        topic_info = model.get_topic_info()
        print(f"✅ Successfully loaded model with {len(topic_info)} topics")
        print("\nSample topics:")
        print(topic_info.head(3))
        return model
    except Exception as e:
        logging.critical(f"Model loading failed: {str(e)}")
        raise RuntimeError(f"Could not load model: {str(e)}")

def process_topics(model, limit=TOPIC_LIMIT):
    """Main processing loop with enhanced error handling"""
    generator = TopicLabelGenerator()
    results = []

    valid_topics = [t for t in model.get_topics() if t != -1][:limit]

    for topic_id in tqdm(valid_topics, desc="Labeling Topics"):
        try:
            # Get topic data
            keywords = [w for w, _ in model.get_topic(topic_id)[:5]]
            docs = model.get_representative_docs(topic_id) or ["No documents available"]

            # Generate label
            label_data = generator.generate_label(keywords, docs)

            # Prepare result entry
            result = {
                "Topic_ID": topic_id,
                "Keywords": ", ".join(keywords),
                **label_data,
                "Sample_Document": docs[0][:200] + "..." if docs else "",
                "Status": "Success"
            }
            results.append(result)

        except Exception as e:
            error_msg = str(e)[:200]  # Truncate long error messages
            logging.error(
                f"Topic {topic_id} processing failed: {error_msg}",
                extra={"topic_id": topic_id}
            )

            # Create error entry with fallback theme
            fallback_theme = f"{keywords[0].capitalize()} and Related Concepts" if 'keywords' in locals() and keywords else "Unknown Topic"
            results.append({
                "Topic_ID": topic_id,
                "Keywords": ", ".join(keywords) if 'keywords' in locals() else "Unknown",
                "Character_Dynamics": "Error",
                "Themes": fallback_theme,  # Always provide a theme even for errors
                "Settings": "Error",
                "Actions_Events": "Error",
                "Emotional_Atmosphere": "Mood - Neutral",
                "Representation": "Portrayal - Neutral",
                "Sample_Document": docs[0][:200] + "..." if 'docs' in locals() and docs else "",
                "Status": f"Error: {error_msg}"
            })
            continue

    return pd.DataFrame(results)

def save_results(df):
    """Save results with timestamp and validation stats"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = f"{OUTPUT_CSV[:-4]}_{timestamp}.csv"

    # Add validation metadata
    success_rate = len(df[df['Status'] == 'Success']) / len(df) if len(df) > 0 else 0
    df['Metadata'] = f"Success rate: {success_rate:.1%} | Generated: {timestamp}"

    df.to_csv(output_path, index=False)

    print(f"\n💾 Saved results to: {output_path}")
    print(f"✅ Success rate: {success_rate:.1%}")

    # Show samples
    if success_rate > 0:
        print("\nSample successful labels:")
        successful = df[df['Status'] == 'Success']
        if len(successful) > 0:
            try:
                print(successful.head(3).to_markdown(index=False))
            except:
                print(successful.head(3))

    return output_path

# ========== MAIN EXECUTION ==========
def main():
    initialize_logging()

    try:
        # Load model
        model = load_model()

        # Process topics
        print(f"\n🚀 Starting to process {TOPIC_LIMIT} topics...")
        start_time = time.time()
        results_df = process_topics(model)

        # Save results
        output_file = save_results(results_df)

        # Final report
        elapsed = time.time() - start_time
        print(f"\n🏁 Process completed in {elapsed:.1f} seconds")
        print(f"📊 Results saved to: {output_file}")
        print(f"📋 Error log: {FAILED_LOG_PATH}")

    except Exception as e:
        logging.critical(f"Pipeline failed: {str(e)}\n{traceback.format_exc()}")
        print(f"⛔ Critical error: {str(e)}")
        exit(1)

if __name__ == "__main__":
    main()


🔍 Loading BERTopic model from: /content/drive/MyDrive/BERTTopic_Models_POS/paraphrase-mpnet-base-v2_Model_1_POS_a/bertopic_model_iter_1_paraphrase-mpnet-base-v2_POS_a.pkl
✅ Successfully loaded model with 364 topics

Sample topics:
   Topic   Count                             Name  \
0     -1  113001  -1_thought_think_thoughts_speak   
1      0    4846   0_smiled_smile_smiles_grinning   
2      1    2847     1_dress_dresses_gowns_attire   

                                      Representation  \
0  [thought, think, thoughts, speak, seemed, say,...   
1  [smiled, smile, smiles, grinning, smirking, gr...   
2  [dress, dresses, gowns, attire, clothing, gown...   

                                               NOUNS  \
0  [hands, way, words, arms, room, body, arm, mom...   
1  [smile, grin, smiles, joke, laughs, chuckle, g...   
2  [dress, hat, shirt, gown, trousers, clothes, j...   

                                 Representative_Docs  
0  [says hard say words inside released hit chest.

Labeling Topics:   0%|          | 0/10 [00:00<?, ?it/s]ERROR:root:Label generation failed: Empty API response
Label generation failed: Empty API response
Label generation failed: Empty API response
Label generation failed: Empty API response
Label generation failed: Empty API response
Label generation failed: Empty API response
Label generation failed: Empty API response
Label generation failed: Empty API response
Label generation failed: Empty API response
ERROR:root:Topic 0 processing failed: Empty API response
Topic 0 processing failed: Empty API response
Topic 0 processing failed: Empty API response
Topic 0 processing failed: Empty API response
Topic 0 processing failed: Empty API response
Topic 0 processing failed: Empty API response
Topic 0 processing failed: Empty API response
Topic 0 processing failed: Empty API response
Topic 0 processing failed: Empty API response
Labeling Topics:  10%|█         | 1/10 [00:00<00:02,  3.38it/s]ERROR:root:Label generation failed: Empty API resp


💾 Saved results to: /content/drive/MyDrive/BERTTopic_Models_POS/verified_labeled_topics_20250329_160747.csv
✅ Success rate: 0.0%

🏁 Process completed in 1.8 seconds
📊 Results saved to: /content/drive/MyDrive/BERTTopic_Models_POS/verified_labeled_topics_20250329_160747.csv
📋 Error log: topic_labeling_errors.log


In [ ]:
import os
import time
import pandas as pd
from bertopic import BERTopic
from tqdm import tqdm
from datetime import datetime
from openai import OpenAI

# 🔐 OpenRouter API Key
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",  # Replace with your actual key
)

# 📁 Path to your saved BERTopic model
model_path = "/content/drive/MyDrive/BERTTopic_Models_POS/paraphrase-mpnet-base-v2_Model_1_POS_a/bertopic_model_iter_1_paraphrase-mpnet-base-v2_POS_a.pkl"

def generate_prompt(topic_words):
    examples = (
        "-1_thought_think_thoughts_speak\tCommunication\tCommunication\tNone\tDialogue and Reflection\n"
        "0_smiled_smile_smiles_grinning\tFriendly Camaraderie\tJoy and Lightheartedness\tJoyous Social Gatherings\tFestive or Cheerful Gatherings\n"
        "1_dress_dresses_gowns_attire\tNone\tFashion & Identity\tDressing Room\tDressing Up\n"
        "2_war_wars_combat_battle\tAllies and Enemies in Battle\tAllies and Enemies in Battle\tWar-Torn Battlefield\tWar and Combat\n"
        "3_door_doors_doorframe_opened\tNone\tOpening & Closing Doors\tEntrances & Corridors\tEntering and Exiting\n"
        "4_married_marriage_wedded_marriages\tMarriage and Commitment\tMarriage and Commitment\tWedding Ceremonies\tEngagements and Weddings\n"
        "5_kiss_kissed_kisses_kissing\tRomantic Intimacy\tRomantic Intimacy\tPrivate Romantic Spaces\tKissing and Embracing\n"
        "6_room_rooms_indoors_downstairs\tDomestic Interactions\tDomestic Interactions\tDomestic Interiors\tEntering and Exiting Rooms\n"
        "7_course_courses_taught_lecture\tTeacher-Student Relationships\tEducation and Knowledge\tAcademic Environment\tTeaching and Learning\n"
        "8_night_tonight_nights_nighttime\tLate Night Interactions\tLate Night Interactions\tNocturnal Setting\tLate Night Interactions\n"
        "9_book_books_bookroom_librarian\tNone\tReading & Studying\tLibraries and Bookshops\tReading and Studying\n"
        # ... You can include the rest if needed; truncated here for brevity
    )
    prompt = f"""
You are an expert in literary and fanfiction topic analysis. For each topic, generate labels for the following columns in exactly this order:
    Name    Character Dynamics    Themes    Settings    Actions & Events    Representation

The output must be a single line with tab-separated values. If a column is not applicable, output "None" for that column.

Below are examples based on manually generated labels for topics produced by this model:
{examples}

Now, given the topic words: {', '.join(topic_words)}, generate the appropriate labels as instructed.
"""
    return prompt

def generate_topic_label(topic_words):
    prompt = generate_prompt(topic_words)
    completion = client.chat.completions.create(
        extra_headers={
            "HTTP-Referer": "<YOUR_SITE_URL>",
            "X-Title": "<YOUR_SITE_NAME>",
        },
        model="deepseek/deepseek-r1",
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.5,
        max_tokens=150,
    )
    return completion.choices[0].message.content.strip()

def verify_and_save_labels(model, output_csv="verified_labeled_topics.csv"):
    topic_info = model.get_topic_info()

    def detect_invalid_labels(label):
        if label is None or label.strip() == "":
            return "❌ Empty Label"
        if "L, a, b, e, l" in label or "Labeling" in label:
            return "⚠️ Garbled Label"
        if "The topic appears to be" in label or "not clear" in label:
            return "⚠️ Placeholder Label"
        return "✅ Valid"

    topic_info["Label_Validity"] = topic_info["Name"].apply(detect_invalid_labels)
    topic_info.to_csv(output_csv, index=False)
    print(f"\n📂 Verified labeled topics saved to: {output_csv}")
    print("\n✅ Sample of Correct Labels:")
    print(topic_info[topic_info["Label_Validity"] == "✅ Valid"][["Topic", "Name"]].head(10))

def main():
    print(f"[{datetime.now()}] 🔄 Loading BERTopic model from: {model_path}")
    model = BERTopic.load(model_path)

    topics = model.get_topics()
    new_labels = {}
    print(f"[{datetime.now()}] 🔄 Generating new labels for {len(topics)} topics...")
    for topic_id, words in tqdm(topics.items(), desc="Processing Topics"):
        topic_words = [word for word, _ in words]
        label = generate_topic_label(topic_words)
        new_labels[topic_id] = label
        time.sleep(1.1)  # Respect API rate limits

    model.set_topic_labels(new_labels)
    updated_model_path = model_path.replace(".pkl", "_labeled.pkl")
    model.save(updated_model_path)
    print(f"[{datetime.now()}] ✅ Updated model with new labels saved as: {updated_model_path}")

    verify_and_save_labels(model, output_csv="/content/drive/MyDrive/BERTTopic_Models_POS/verified_labeled_topics.csv")

if __name__ == "__main__":
    main()

[2025-03-28 19:41:53.391031] 🔄 Loading BERTopic model from: /content/drive/MyDrive/BERTTopic_Models_POS/paraphrase-mpnet-base-v2_Model_1_POS_a/bertopic_model_iter_1_paraphrase-mpnet-base-v2_POS_a.pkl
[2025-03-28 19:42:00.319777] 🔄 Generating new labels for 364 topics...


Processing Topics: 100%|██████████| 364/364 [1:24:18<00:00, 13.90s/it]
2025-03-28 21:06:18,491 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


[2025-03-28 21:07:13.917505] ✅ Updated model with new labels saved as: /content/drive/MyDrive/BERTTopic_Models_POS/paraphrase-mpnet-base-v2_Model_1_POS_a/bertopic_model_iter_1_paraphrase-mpnet-base-v2_POS_a_labeled.pkl

📂 Verified labeled topics saved to: /content/drive/MyDrive/BERTTopic_Models_POS/verified_labeled_topics.csv

✅ Sample of Correct Labels:
   Topic                                 Name
0     -1      -1_thought_think_thoughts_speak
1      0       0_smiled_smile_smiles_grinning
2      1         1_dress_dresses_gowns_attire
3      2             2_war_wars_combat_battle
4      3        3_door_doors_doorframe_opened
5      4  4_married_marriage_wedded_marriages
6      5         5_kiss_kissed_kisses_kissing
7      6      6_room_rooms_indoors_downstairs
8      7      7_course_courses_taught_lecture
9      8     8_night_tonight_nights_nighttime


In [ ]:
from bertopic import BERTopic

In [ ]:
import os
import time
import requests
import pandas as pd
from bertopic import BERTopic
from tqdm import tqdm
from datetime import datetime

# ✅ Your Together AI API Key
TOGETHER_API_KEY = "4a24bea11227b835632c6d87732fcea473561ecad2b3c0a20cd38360d45d830e"

# Path to the specific restored model
model_path = "/content/drive/MyDrive/BERTTopic_Models_POS/paraphrase-mpnet-base-v2_Model_1_POS_a/bertopic_model_iter_1_paraphrase-mpnet-base-v2_POS_a.pkl"


def generate_prompt(topic_words):
    # Define the manual examples as a multi-line string with tab-separated values.
    examples = (
        "-1_thought_think_thoughts_speak\tCommunication\tCommunication\tNone\tDialogue and Reflection\n"
        "0_smiled_smile_smiles_grinning\tFriendly Camaraderie\tJoy and Lightheartedness\tJoyous Social Gatherings\tFestive or Cheerful Gatherings\n"
        "1_dress_dresses_gowns_attire\tNone\tFashion & Identity\tDressing Room\tDressing Up\n"
        "2_war_wars_combat_battle\tAllies and Enemies in Battle\tAllies and Enemies in Battle\tWar-Torn Battlefield\tWar and Combat\n"
        "3_door_doors_doorframe_opened\tNone\tOpening & Closing Doors\tEntrances & Corridors\tEntering and Exiting\n"
        "4_married_marriage_wedded_marriages\tMarriage and Commitment\tMarriage and Commitment\tWedding Ceremonies\tEngagements and Weddings\n"
        "5_kiss_kissed_kisses_kissing\tRomantic Intimacy\tRomantic Intimacy\tPrivate Romantic Spaces\tKissing and Embracing\n"
        "6_room_rooms_indoors_downstairs\tDomestic Interactions\tDomestic Interactions\tDomestic Interiors\tEntering and Exiting Rooms\n"
        "7_course_courses_taught_lecture\tTeacher-Student Relationships\tEducation and Knowledge\tAcademic Environment\tTeaching and Learning\n"
        "8_night_tonight_nights_nighttime\tLate Night Interactions\tLate Night Interactions\tNocturnal Setting\tLate Night Interactions\n"
        "9_book_books_bookroom_librarian\tNone\tReading & Studying\tLibraries and Bookshops\tReading and Studying\n"
        "10_breath_breaths_breathed_breathes\tNone\tBreathing & Gasping\tNone\tBreathing & Gasping\n"
        "11_stared_gazes_gazed_gaze\tSilent Communication\tStaring & Gazing\tNone\tIntense Staring\n"
        "12_crying_cry_cries_sobbed\tEmotional Breakdowns\tPain and Loss\tNone\tExpressing Sorrow\n"
        "13_women_females_feminist_womanhood\tDebating Gender Roles\tGender and Sexuality\tNone\tChallenging Gender Norms\n"
        "14_money_financial_cash_pays\tFinancial Ties\tWealth and Economics\tNone\tFinancial Transactions\n"
        "15_mouth_mouths_lips_tongue\tCommunication & Expression\tSpeaking, Biting & Licking\tNone\tSpeaking, Biting & Licking\n"
        "16_asleep_sleepily_slept_sleeping\tSleeping and Awakening\tSleeping and Awakening\tSleeping Quarters\tSlumber and Wakefulness\n"
        "17_conversation_conversational_conversations_conversationist\tConversation and Discussion\tConversation and Discussion\tNone\tConversation and Discussion\n"
        "18_nodded_nods_nod_agreed\tNonverbal Communication\tAcknowledgment and Agreement\tNone\tAgreement and Acknowledgment\n"
        "19_blood_bloodier_bloodied_bloodstains\tInjury and Violence\tInjury and Violence\tBloody or Violent Scenes\tInjury and Violence\n"
        "20_light_lightness_lights_illuminated\tNone\tIlluminated Spaces\tBrightly Lit Spaces\tLighting & Shining\n"
        "21_chair_chairs_seat_armchair\tCasual Encounters\tEveryday Domestic Details\tEveryday Domestic Details\tTaking a Seat\n"
        "22_coldness_cold_chilly_colder\tNone\tFrosty Landscapes\tFrosty Landscapes\tShivering & Freezing\n"
        "23_ship_boat_sailed_vessel\tCrew Interactions\tSailing and Voyaging\tShips and the Sea\tSailing and Voyaging\n"
        "24_ca_california_act_see\tNone\tUncertainty and Possibility\tNone\tNone\n"
        "25_wine_wines_drank_sipped\tToasting and Drinking\tToasting and Drinking\tTaverns or Banquets\tDrinking and Toasting\n"
        "26_window_windowpane_windows_open\tNone\tLooking Outside\tWindows and Balconies\tWindow Gazing and Observation\n"
        "27_horse_horsewhipped_horseman_horseback\tNone\tHorseback Riding\tEquestrian and Rural Paths\tRiding & Galloping\n"
        "28_kill_kills_killing_murdering\tViolent Conflict\tActs of Violence\tCrime or Battle Scenes\tMurder and Violence\n"
        "29_garden_gardens_gardenia_gardening\tNone\tPlanting and Gardening\tLush Gardens\tPlanting and Gardening\n"
        "30_stairs_staircase_staircases_stairway\tNone\tMoving Up or Down Staircases\tStaircases and Corridors\tMoving Up or Down Staircases\n"
        "31_dinner_dinners_dining_meal\tShared Meals and Social Bonding\tShared Meals and Social Bonding\tDining Rooms or Restaurants\tEating and Feasting\n"
        "32_bathroom_bathrooms_washroom_restroom\tNone\tBathrooms and Washrooms\tBathrooms and Washrooms\tBathing and Cleaning\n"
        "33_letter_wrote_letters_received\tPostal or Digital Messaging\tPostal or Digital Messaging\tNone\tWriting and Sending Letters and Emails\n"
        "34_anger_angry_rage_angered\tOutbursts and Accusations\tWrath and Frustration\tNone\tOutbursts and Accusations\n"
        "35_hurry_hurried_hastened_hastening\tNone\tRushing and Hastening\tNone\tRushing and Hastening\n"
        "36_memory_remembering_memories_remembers\tNone\tMemory and Nostalgia\tNone\tRecollection and Reminiscence\n"
        "37_feelings_emotions_emotion_emotional\tNone\tComplex Emotions and Uncertainty\tNone\tExperiencing and Expressing Emotions\n"
        "38_sort_sorts_sorting_sorted\tNone\tSorting and Categorizing\tNone\tSorting and Categorizing\n"
        "39_town_townsfolk_towns_village\tNone\tTowns or Villages and Neighbors\tTowns or Villages\tNone\n"
        "40_home_homely_homed_homesick\tNone\tReturning or Homecoming\tHome and Domestic Spaces\tReturning or Homecoming\n"
        "41_tree_trees_grove_forest\tNone\tForests and Woodlands\tForests and Woodlands\tNone\n"
        "42_kitchen_kitchens_kitchenette_countertop\tNone\tCooking and Cleaning\tKitchens\tCooking and Cleaning\n"
        "43_walked_walking_walks_walk\tNone\tWalking and Strolling\tRoads and Paths\tWalking and Strolling\n"
        "44_wish_wishes_wished_wishing\tNone\tWishing and Hoping\tNone\tWishing and Hoping\n"
        "45_cup_teacup_cups_coffee\tNone\tDrinking Coffee or Tea\tCafés or Tea Rooms\tDrinking and Coffee or Tea\n"
        "46_pistol_holstered_pistols_revolvers\tWeaponized Confrontations\tWeaponized Confrontations\tArmed Confrontations\tFiring and Aiming\n"
        "47_flames_flame_blazes_blazing\tNone\tIgniting and Burning\tBurning Landscapes\tIgniting and Burning\n"
        "48_kindness_gratefulness_kindest_generosity\tCaring Interactions\tCaring Interactions\tNone\tCaring and Expressing Gratitude\n"
        "49_carriage_carriages_wagon_cart\tNone\tRoads and Carriages\tRoads and Carriages\tTravelling\n"
        "50_game_games_played_play\tCompetitive Interactions\tPlay and Competition\tRecreational or Gaming Areas\tPlaying and Competing\n"
        "51_whispered_whispers_whisper_murmured\tSecrets and Soft Communication\tSecrets and Soft Communication\tQuiet, Private Spaces\tWhispering and Murmuring\n"
        "52_cake_cakes_desserts_baking\tNone\tBaking and Serving Desserts\tBakeries or Dining Tables\tBaking and Serving Desserts\n"
        "53_phone_phones_cellphone_smartphone\tDigital Communication\tDigital Communication\tModern Tech Environments\tCalling and Texting\n"
        "54_table_tables_tablecloth_tableau\tNone\tSetting and Arranging Tables\tRestaurants\tSetting and Arranging Tables\n"
        "55_stomach_stomachs_tummy_gut\tNone\tNone\tNone\tNone\n"
        "56_dancing_dancefloor_dances_danced\tDancing and Moving\tDancing and Moving\tDance Halls or Clubs\tDancing and Moving\n"
        "57_shook_shaken_shaking_trembled\tNone\tShaking and Trembling\tChaotic or Unstable Environments\tShaking and Trembling\n"
        "58_name_surname_naming_names\tNaming and Introducing\tNaming and Introducing\tSocial or Formal Introductions\tNaming and Introducing\n"
        "59_leg_legs_knees_knee\tNone\tNone\tNone\tNone\n"
        "60_months_month_weeks_monthly\tNone\tPassage of Time\tTemporal or Calendar Contexts\tMarking Time\n"
        "61_leave_leaving_depart_departure\tParting or Farewell Dynamics\tParting or Farewell Dynamics\tNone\tLeaving or Departing\n"
        "62_medical_hospital_physicians_physician\tPatient–Doctor Interactions\tHealth and Healing\tHospitals and Clinics\tTreatment and Surgery\n"
        "63_trust_trusts_trustworthiness_trusting\tTrust and Betrayal\tTrust and Betrayal\tNone\tTrusting and Discerning\n"
        "64_scent_smelling_scents_scenting\tNone\tAromas and Sensory Experience\tNone\tSmelling and Inhaling\n"
        "65_minutes_minute_hour_seconds\tNone\tTime and Duration\tNone\tCounting and Measuring Time\n"
        "66_yes_yeah_sure_ok\tAffirmative Exchanges\tAgreement and Confirmation\tConversational Settings\tResponding and Confirming\n"
        "67_seat_seated_seats_sit\tNone\tNone\tNone\tNone\n"
        "68_believe_believes_belief_believed\tNone\tFaith and Conviction\tIntellectual or Spiritual Contexts\tBelieving and Considering\n"
        "69_bag_bags_handbag_luggage\tNone\tPacking and Carrying\tTravel or Shopping Contexts\tPacking and Carrying\n"
        "70_train_trains_railway_railroad\tNone\tRailway Stations and Locomotives\tRailway Stations and Locomotives\tBoarding and Traveling by Train\n"
        "71_parents_parent_grandparents_fathers\tNone\tParenthood and Kinship\tHome or Family Gatherings\tNone\n"
        "72_curls_curled_hairs_whiskers\tNone\tStyling and Combing\tBeauty or Grooming Spaces\tStyling and Combing\n"
        "73_eyebrow_eyebrows_glanced_brow\tSubtle Interactions\tSkepticism and Curiosity\tFace-to-Face Encounters\tRaising and Questioning\n"
        "74_hug_hugs_hugged_hugging\tAffectionate Connections\tComfort and Support\tIntimate or Friendly Spaces\tHugging and Embracing\n"
        "75_heard_hears_overheard_hearing\tHearing and Listening\tHearing and Listening\tAuditory or Conversational Spaces\tHearing and Listening\n"
        "76_towel_washcloth_cleans_towels\tNone\tHygiene and Cleanliness\tBathrooms or Laundry Rooms\tWiping and Cleaning\n"
        "77_hate_hatefully_hating_hateful\tHostility and Disdain\tHostility and Disdain\tNone\tExpressing Hatred\n"
        "78_eat_eating_eaten_food\tNone\tNourishment and Appetite\tDining or Kitchen Spaces\tEating and Consuming\n"
        "79_mask_masks_masked_disguise\tNone\tDisguise and Concealment\tCostume or Masquerade Settings\tMasking and Hiding\n"
        "80_mirror_mirrors_reflected_mirroring\tNone\tNone\tDressing Rooms or Bathrooms\tNone\n"
        "81_case_cases_circumstances_trial\tInvestigative Partners\tInvestigating and Solving Cases\tCourtrooms and Investigation Scenes\tInvestigating and Solving Cases\n"
        "82_rain_rained_raining_rains\tNone\tStorming and Pouring Rain\tRainy Outdoors\tStorming and Pouring Rain\n"
        "83_lie_lied_lies_liar\tDishonest Relationships\tTruth vs. Deception\tNone\tLying and Deceiving\n"
        "84_cock_cocks_cockhead_cockstand\tSexual Intimacy\tDesire and Sexuality\tPrivate, Intimate Spaces\tSexual Acts and Arousal\n"
        "85_apologize_apologizes_apologized_apology\tReconciliation Attempts\tApologizing and Forgiving\tInterpersonal Conversations\tApologizing and Forgiving\n"
        "86_dogs_canines_pets_pups\tHuman-Animal Companionship\tCaring for Pets\tHomes and Outdoors\tCaring for Pets\n"
        "87_pregnancy_birth_pregnant_birthing\tNone\tGiving Birth and Expecting Children\tHomes and Birthplaces\tGiving Birth and Expecting Children\n"
        "88_safe_safest_safes_safekeeping\tProtective Relationships\tSafety and Security\tNone\tEnsuring Safety\n"
        "89_party_parties_festivities_guests\tSocializing and Friendships\tParties and Social Gatherings\tParties and Social Gatherings\tCelebrating and Entertaining\n"
        "90_shoulder_shoulders_neck_back\tConsoling and Supporting\tEmpathy and Support\tQuiet, Intimate Spaces\tConsoling and Supporting\n"
        "91_business_dealings_companies_firm\tProfessional and Business Relationships\tCommerce and Negotiation\tOffices and Marketplaces\tConducting Business\n"
        "92_feet_legs_footfalls_footstep\tNone\tWalking and Traveling\tRoads, Paths, and Streets\tWalking and Traveling\n"
        "93_fact_facts_incontrovertible_corroborated\tNone\tProving and Establishing Facts\tIntellectual Discussions\tProving and Establishing Facts\n"
        "94_piano_pianist_pianoforte_pianos\tNone\tMusical Performance\tConcerts and Salons\tPlaying Instruments\n"
        "95_crowd_crowds_onlookers_crowding\tSpectacle and Group Dynamics\tSpectacle and Group Dynamics\tPublic Places and Events\tWatching and Gathering\n"
        "96_birds_birders_birding_birdsong\tNone\tBird Watching\tOutdoor, Nature\tBird Watching\n"
        "97_throat_cleared_clears_swallowing\tNone\tCommunication and Awkwardness\tNone\tClearing Throat and Pausing\n"
        "98_afraid_fearful_feared_scared\tNone\tVulnerability and Anxiety\tThreatening Situations\tExperiencing Fear\n"
        "99_paper_ink_papers_penmanship\tNone\tWriting and Recording\tStudies\tWriting and Recording\n"
        "100_waves_wave_surf_ocean\tNone\tSwimming and Surfing\tBeaches and Oceansides\tSwimming and Surfing\n"
        "101_police_policemen_policeman_cops\tLaw Enforcement Encounters\tOrder and Justice\tCrime Scenes and Stations\tArresting and Patrolling\n"
        "102_singing_sings_sang_sung\tNone\tMusical Performances\tConcerts and Gatherings\tSinging and Performing\n"
        "103_friendship_friendships_befriend_companionship\tFriendly Bonds\tCompanionship and Trust\tNone\tBefriending and Bonding\n"
        "104_chance_chances_maybe_hoping\tNone\tTaking Chances\tUnpredictable Situations\tTaking Chances\n"
        "105_done_enough_finished_finish\tNone\tFinishing and Completing\tWorkspaces and Tasks\tFinishing and Completing\n"
        "106_bedroom_bedrooms_bedchamber_bedchambers\tNone\tBedrooms and Sleeping Areas\tBedrooms and Sleeping Areas\tNone\n"
        "107_dreams_dreaming_dreamed_dreamt\tNone\tBedrooms and Sleeping Areas\tBedrooms and Sleeping Areas\tNone\n"
        "108_pillow_pillows_pillowcase_mattress\tNone\tResting and Sleeping\tBedrooms and Sleeping Areas\tResting and Sleeping\n"
        "109_cigarette_cigarettes_smokes_smoked\tNone\tSmoking and Puffing\tNone\tSmoking and Puffing\n"
        "110_danger_dangers_dangerous_threat\tNone\tPeril and Caution\tPeril Situations\tFacing Danger"
    )
    prompt = f"""
You are an expert in literary and fanfiction topic analysis. For each topic, generate labels for the following columns in exactly this order:
    Name    Character Dynamics    Themes    Settings    Actions & Events    Representation

The output must be a single line with tab-separated values. If a column is not applicable, output "None" for that column.

Below are examples based on manually generated labels for topics produced by this model:
{examples}

Now, given the topic words: {', '.join(topic_words)}, generate the appropriate labels as instructed.
"""
    return prompt


def generate_topic_label(topic_words):
    prompt = generate_prompt(topic_words)
    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "mistralai/Mistral-7B-Instruct-v0.1",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.5,
        "max_tokens": 150
    }
    while True:
        try:
            response = requests.post("https://api.together.xyz/v1/chat/completions", json=payload, headers=headers)
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"].strip()
            elif response.status_code == 429:
                print("⚠️ Rate limit hit! Waiting 2 seconds before retrying...")
                time.sleep(2)
            else:
                return "Unknown Topic"
        except requests.exceptions.RequestException:
            return "Unknown Topic"


def verify_and_save_labels(model, output_csv="verified_labeled_topics.csv"):
    topic_info = model.get_topic_info()

    def detect_invalid_labels(label):
        if label is None or label.strip() == "":
            return "❌ Empty Label"
        if "L, a, b, e, l" in label or "Labeling" in label:
            return "⚠️ Garbled Label"
        if "The topic appears to be" in label or "not clear" in label:
            return "⚠️ Placeholder Label"
        return "✅ Valid"

    topic_info["Label_Validity"] = topic_info["Name"].apply(detect_invalid_labels)
    topic_info.to_csv(output_csv, index=False)
    print(f"\n📂 Verified labeled topics saved to: {output_csv}")
    print("\n✅ Sample of Correct Labels:")
    print(topic_info[topic_info["Label_Validity"] == "✅ Valid"][["Topic", "Name"]].head(10))


def main():
    print(f"[{datetime.now()}] 🔄 Loading BERTopic model from: {model_path}")
    model = BERTopic.load(model_path)

    topics = model.get_topics()
    new_labels = {}
    print(f"[{datetime.now()}] 🔄 Generating new labels for {len(topics)} topics...")
    for topic_id, words in tqdm(topics.items(), desc="Processing Topics"):
        topic_words = [word for word, _ in words]
        label = generate_topic_label(topic_words)
        new_labels[topic_id] = label
        time.sleep(1.1)  # Enforce API rate limit
    model.set_topic_labels(new_labels)

    updated_model_path = model_path.replace(".pkl", "_labeled.pkl")
    model.save(updated_model_path)
    print(f"[{datetime.now()}] ✅ Updated model with new labels saved as: {updated_model_path}")

    verify_and_save_labels(model, output_csv="/content/drive/MyDrive/BERTTopic_Models_POS/verified_labeled_topics.csv")


if __name__ == "__main__":
    main()

[2025-03-22 19:53:21.148195] 🔄 Loading BERTopic model from: /content/drive/MyDrive/BERTTopic_Models_POS/paraphrase-mpnet-base-v2_Model_1_POS_a/bertopic_model_iter_1_paraphrase-mpnet-base-v2_POS_a.pkl
[2025-03-22 19:54:20.129804] 🔄 Generating new labels for 364 topics...


Processing Topics: 100%|██████████| 364/364 [13:48<00:00,  2.28s/it]
2025-03-22 20:08:08,435 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


[2025-03-22 20:08:17.239159] ✅ Updated model with new labels saved as: /content/drive/MyDrive/BERTTopic_Models_POS/paraphrase-mpnet-base-v2_Model_1_POS_a/bertopic_model_iter_1_paraphrase-mpnet-base-v2_POS_a_labeled.pkl

📂 Verified labeled topics saved to: /content/drive/MyDrive/BERTTopic_Models_POS/verified_labeled_topics.csv

✅ Sample of Correct Labels:
   Topic                                 Name
0     -1      -1_thought_think_thoughts_speak
1      0       0_smiled_smile_smiles_grinning
2      1         1_dress_dresses_gowns_attire
3      2             2_war_wars_combat_battle
4      3        3_door_doors_doorframe_opened
5      4  4_married_marriage_wedded_marriages
6      5         5_kiss_kissed_kisses_kissing
7      6      6_room_rooms_indoors_downstairs
8      7      7_course_courses_taught_lecture
9      8     8_night_tonight_nights_nighttime


In [ ]:
# =====================================
# 1. Import Libraries
# =====================================
import nltk
nltk.download('punkt_tab')
import os
import time
import logging
import pandas as pd
import numpy as np
import re
from datetime import datetime
import torch
import gc
from tqdm import tqdm
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

# Import RAPIDS' UMAP and HDBSCAN
from cuml.manifold import UMAP  # GPU-accelerated UMAP
from cuml.cluster import HDBSCAN  # GPU-accelerated HDBSCAN
import cupy as cp  # For GPU arrays

# Import NLTK modules for text processing
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Import BERTopic's representation models
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, PartOfSpeech

# Import json module
import json

# Import string module for punctuation handling
import string

# =====================================
# 2. Configure Logging
# =====================================

# Define logging configuration
LOG_FILENAME = 'bertopic_training.log'

# Create a custom logger
logger = logging.getLogger('BERTopic_Training')
logger.setLevel(logging.DEBUG)

# Prevent adding multiple handlers in environments like Google Colab
if not logger.handlers:
    # Create handlers
    c_handler = logging.StreamHandler()
    f_handler = logging.FileHandler(LOG_FILENAME)
    c_handler.setLevel(logging.INFO)
    f_handler.setLevel(logging.DEBUG)

    # Create formatters and add them to handlers
    c_format = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    f_format = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    c_handler.setFormatter(c_format)
    f_handler.setFormatter(f_format)

    # Add handlers to the logger
    logger.addHandler(c_handler)
    logger.addHandler(f_handler)

# =====================================
# 3. Check and Load SpaCy Model for POS Tagging
# =====================================

# Load spaCy's English model for POS tagging, with error handling if the model is missing
import spacy
from spacy.cli import download as spacy_download

try:
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
except OSError:
    logger.info("Downloading 'en_core_web_sm' model for spaCy as it was not found...")
    spacy_download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# =====================================
# 4. Define Paths and Load Stop Words
# =====================================

# Define the paths for datasets, models, and resources
dataset_path = '/content/drive/MyDrive/fanfiction_project_data_and_models/processed_novels_sentences_new.csv'
additional_stop_words_characters_names = '/content/drive/MyDrive/fanfiction_project_data_and_models/fanfiction_additional_stop_list_character_names.txt'

def preprocess_stopwords(stopwords_file_path):
    """
    Preprocess the stop words list by lowercasing, removing punctuation, and splitting multi-word entries.
    """
    try:
        with open(stopwords_file_path, 'r', encoding='utf-8') as file:
            raw_stop_words = file.read().splitlines()

        processed_stop_words = set()
        translator = str.maketrans('', '', string.punctuation)

        for stop_word in raw_stop_words:
            stop_word = stop_word.lower().translate(translator)
            words = stop_word.split()
            for word in words:
                if word:
                    processed_stop_words.add(word)

        logger.info(f"Processed stop words count: {len(processed_stop_words)}")
        return processed_stop_words

    except Exception as e:
        logger.error(f"Error processing stop words: {e}")
        return set(stopwords.words('english'))  # Fallback to NLTK's stop words

logger.info("Loading and preprocessing additional stop words...")
custom_stop_words = preprocess_stopwords(additional_stop_words_characters_names)

# Combine with NLTK's stop words
stop_words = set(stopwords.words('english'))
stop_words.update(custom_stop_words)
logger.info(f"Total stop words after preprocessing: {len(stop_words)}")

# =====================================
# 5. Load and Preprocess the Dataset
# =====================================

def load_dataset(path, stop_words, test_mode=False, sample_size=10000, chunksize=None):
    """
    Load and preprocess the dataset.
    """
    logger.info("Loading and preprocessing dataset...")
    start_time = time.time()

    try:
        if test_mode:
            logger.info(f"Test mode enabled. Sampling {sample_size} sentences from the dataset.")
            if chunksize is None:
                chunksize = 1000
            df_iter = pd.read_csv(path, chunksize=chunksize)
            sampled_chunks = []
            total_sampled = 0
            for chunk in df_iter:
                remaining = sample_size - total_sampled
                if remaining <= 0:
                    break
                n_samples = min(remaining, len(chunk))
                sampled = chunk.sample(n=n_samples, random_state=42)
                sampled_chunks.append(sampled)
                total_sampled += n_samples
            df = pd.concat(sampled_chunks) if sampled_chunks else pd.DataFrame()
        else:
            df = pd.read_csv(path)

        df['Sentence'] = df['Sentence'].astype(str).apply(lambda x: re.sub(r'\n+', ' ', x))
        df['Sentence'] = df['Sentence'].str.replace(r'\s+', ' ', regex=True).str.strip().str.lower()

        processed_docs = []
        for sentence in df['Sentence']:
            tokens = word_tokenize(sentence)
            tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
            processed_sentence = ' '.join(tokens)
            processed_docs.append(processed_sentence)

        corpus_filtered = [sentence for sentence in processed_docs if len(sentence.split()) >= 3]
        logger.info(f"Filtered corpus size: {len(corpus_filtered)}")
        logger.info(f"Time taken for loading and preprocessing: {time.time() - start_time:.2f} seconds")
        return corpus_filtered

    except Exception as e:
        logger.error(f"Failed to load and preprocess dataset: {e}")
        return []

# =====================================
# 6. Create Parameters DataFrame
# =====================================

import pandas as pd

import pandas as pd

def create_dataframe():
    """
    Create the dataframe containing the top 10 models (new version) without the extra columns.
    """
    models = [
        {
            "Embeddings_Model": "paraphrase-mpnet-base-v2",
            "Iteration": 1,
            "Coherence": 0.488537,
            "Topic_Diversity": 0.986667,
            "bertopic__min_topic_size": 57,
            "bertopic__top_n_words": 37,
            "hdbscan__min_cluster_size": 132,
            "hdbscan__min_samples": 57,
            "umap__min_dist": 0.053015,
            "umap__n_components": 4,
            "umap__n_neighbors": 39,
            "vectorizer__min_df": 0.004806
        },
        {
            "Embeddings_Model": "paraphrase-MiniLM-L6-v2",
            "Iteration": 1,
            "Coherence": 0.416477,
            "Topic_Diversity": 1.000000,
            "bertopic__min_topic_size": 57,
            "bertopic__top_n_words": 37,
            "hdbscan__min_cluster_size": 132,
            "hdbscan__min_samples": 57,
            "umap__min_dist": 0.053015,
            "umap__n_components": 4,
            "umap__n_neighbors": 39,
            "vectorizer__min_df": 0.004806
        },
        {
            "Embeddings_Model": "all-MiniLM-L12-v2",
            "Iteration": 101,
            "Coherence": 0.392483,
            "Topic_Diversity": 0.975000,
            "bertopic__min_topic_size": 127,
            "bertopic__top_n_words": 26,
            "hdbscan__min_cluster_size": 128,
            "hdbscan__min_samples": 68,
            "umap__min_dist": 0.066224,
            "umap__n_components": 2,
            "umap__n_neighbors": 20,
            "vectorizer__min_df": 0.002257
        },
        {
            "Embeddings_Model": "all-MiniLM-L12-v2",
            "Iteration": 66,
            "Coherence": 0.419094,
            "Topic_Diversity": 0.825000,
            "bertopic__min_topic_size": 228,
            "bertopic__top_n_words": 27,
            "hdbscan__min_cluster_size": 105,
            "hdbscan__min_samples": 22,
            "umap__min_dist": 0.011738,
            "umap__n_components": 2,
            "umap__n_neighbors": 41,
            "vectorizer__min_df": 0.006573
        },
        {
            "Embeddings_Model": "paraphrase-MiniLM-L6-v2",
            "Iteration": 112,
            "Coherence": 0.379404,
            "Topic_Diversity": 0.937500,
            "bertopic__min_topic_size": 131,
            "bertopic__top_n_words": 22,
            "hdbscan__min_cluster_size": 86,
            "hdbscan__min_samples": 73,
            "umap__min_dist": 0.083975,
            "umap__n_components": 2,
            "umap__n_neighbors": 3,
            "vectorizer__min_df": 0.001335
        },
        {
            "Embeddings_Model": "paraphrase-MiniLM-L6-v2",
            "Iteration": 59,
            "Coherence": 0.400154,
            "Topic_Diversity": 0.761755,
            "bertopic__min_topic_size": 26,
            "bertopic__top_n_words": 38,
            "hdbscan__min_cluster_size": 120,
            "hdbscan__min_samples": 95,
            "umap__min_dist": 0.052454,
            "umap__n_components": 2,
            "umap__n_neighbors": 2,
            "vectorizer__min_df": 0.007219
        },
        {
            "Embeddings_Model": "paraphrase-mpnet-base-v2",
            "Iteration": 38,
            "Coherence": 0.323946,
            "Topic_Diversity": 1.000000,
            "bertopic__min_topic_size": 187,
            "bertopic__top_n_words": 35,
            "hdbscan__min_cluster_size": 60,
            "hdbscan__min_samples": 88,
            "umap__min_dist": 0.011252,
            "umap__n_components": 4,
            "umap__n_neighbors": 42,
            "vectorizer__min_df": 0.009032
        },
        {
            "Embeddings_Model": "paraphrase-MiniLM-L6-v2",
            "Iteration": 13,
            "Coherence": 0.358554,
            "Topic_Diversity": 0.820319,
            "bertopic__min_topic_size": 59,
            "bertopic__top_n_words": 16,
            "hdbscan__min_cluster_size": 54,
            "hdbscan__min_samples": 74,
            "umap__min_dist": 0.020023,
            "umap__n_components": 4,
            "umap__n_neighbors": 11,
            "vectorizer__min_df": 0.002484
        },
        {
            "Embeddings_Model": "all-MiniLM-L12-v2",
            "Iteration": 60,
            "Coherence": 0.366164,
            "Topic_Diversity": 0.761207,
            "bertopic__min_topic_size": 77,
            "bertopic__top_n_words": 13,
            "hdbscan__min_cluster_size": 144,
            "hdbscan__min_samples": 51,
            "umap__min_dist": 0.069807,
            "umap__n_components": 4,
            "umap__n_neighbors": 2,
            "vectorizer__min_df": 0.002135
        },
        {
            "Embeddings_Model": "all-MiniLM-L12-v2",
            "Iteration": 52,
            "Coherence": 0.331452,
            "Topic_Diversity": 0.900000,
            "bertopic__min_topic_size": 245,
            "bertopic__top_n_words": 38,
            "hdbscan__min_cluster_size": 98,
            "hdbscan__min_samples": 30,
            "umap__min_dist": 0.044896,
            "umap__n_components": 9,
            "umap__n_neighbors": 28,
            "vectorizer__min_df": 0.005585
        }
    ]
    return pd.DataFrame(models)


params_df = create_dataframe()

# =====================================
# 7. Load Embedding Models
# =====================================

def load_embedding_models(model_names):
    embedding_models = {}
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    logger.info(f"Using device for embeddings: {device}")

    for model_name in model_names:
        try:
            embedding_model = SentenceTransformer(model_name, device=device)
            embedding_models[model_name] = embedding_model
            logger.info(f"Model {model_name} loaded successfully.")
        except Exception as e:
            logger.error(f"Failed to load embedding model {model_name}: {e}")
    return embedding_models

embedding_model_names = params_df['Embeddings_Model'].unique()
embedding_models = load_embedding_models(embedding_model_names)

# =====================================
# 8. Define POS Configurations
# =====================================

def get_representation_model(pos_configuration):
    """
    Return the representation model based on POS configuration.
    """
    main_representation = KeyBERTInspired(top_n_words=30)
    if pos_configuration == 'a':
        # a) ONLY NOUNS
        pos_nouns = PartOfSpeech(pos_patterns=[[{"POS": "NOUN"}]])
        return {"Main": main_representation, "NOUNS": pos_nouns}
    elif pos_configuration == 'b':
        # b) NOUNS AND ADJECTIVES
        pos_nouns = PartOfSpeech(pos_patterns=[[{"POS": "NOUN"}]])
        pos_adjectives = PartOfSpeech(pos_patterns=[[{"POS": "ADJ"}]])
        return {"Main": main_representation, "NOUNS": pos_nouns, "ADJECTIVES": pos_adjectives}
    elif pos_configuration == 'c':
        # c) NOUNS AND VERBS
        pos_nouns = PartOfSpeech(pos_patterns=[[{"POS": "NOUN"}]])
        pos_verbs = PartOfSpeech(pos_patterns=[[{"POS": "VERB"}]])
        return {"Main": main_representation, "NOUNS": pos_nouns, "VERBS": pos_verbs}
    else:
        raise ValueError(f"Invalid POS configuration: {pos_configuration}")

# =====================================
# 9. Train and Save BERTopic Models
# =====================================

def train_and_save_models(corpus, params_df, embedding_models, stop_words):
    main_output_dir = "/content/drive/MyDrive/BERTTopic_Models_POS"
    os.makedirs(main_output_dir, exist_ok=True)

    for idx, row in tqdm(params_df.iterrows(), total=params_df.shape[0], desc="Training Models"):
        embedding_model_name = row['Embeddings_Model']
        iteration = row['Iteration']
        embedding_model = embedding_models.get(embedding_model_name)
        if embedding_model is None:
            logger.error(f"Embedding model {embedding_model_name} not found.")
            continue

        for pos_config in ['a', 'b', 'c']:
            try:
                representation_model = get_representation_model(pos_config)
            except ValueError as e:
                logger.error(e)
                continue

            try:
                topic_model = BERTopic(
                    embedding_model=embedding_model,
                    representation_model=representation_model,
                    top_n_words=row['bertopic__top_n_words'],
                    min_topic_size=row['bertopic__min_topic_size'],
                    language='english'
                )
                topics, probs = topic_model.fit_transform(corpus)
            except Exception as e:
                logger.error(f"Error during training: {e}")
                continue

            safe_embedding = embedding_model_name.replace('/', '_')
            model_output_dir = os.path.join(main_output_dir, f"{safe_embedding}_Model_{idx+1}_POS_{pos_config}")
            os.makedirs(model_output_dir, exist_ok=True)

            model_filename = f"bertopic_model_iter_{iteration}_{safe_embedding}_POS_{pos_config}.pkl"
            model_path = os.path.join(model_output_dir, model_filename)
            topic_model.save(model_path)

            try:
                topics_info = topic_model.get_topic_info()
                topics_dict = topic_model.get_topics()

                # Save topics information to CSV file
                csv_filename = f"topics_info_iter_{iteration}_POS_{pos_config}.csv"
                topics_info.to_csv(os.path.join(model_output_dir, csv_filename), index=False)

                # Prepare topics data for JSON serialization with float conversion
                topics_json = {}
                for topic_num, words in topics_dict.items():
                    if topic_num == -1:  # Skip outlier topic
                        continue

                    # Convert probabilities to standard floats for JSON serialization
                    filtered_words = [{word: float(prob)} for word, prob in words if word.strip()]
                    if filtered_words:  # Ensure we only add non-empty topics
                        topics_json[str(topic_num)] = filtered_words

                # Save topics to JSON file
                json_filename = f"topics_iter_{iteration}_POS_{pos_config}.json"
                json_path = os.path.join(model_output_dir, json_filename)
                with open(json_path, 'w') as json_file:
                    json.dump(topics_json, json_file, indent=4)
                logger.info(f"Topics JSON saved to {json_path}")

            except Exception as e:
                logger.error(f"Failed to save topics: {e}")

            del topic_model
            gc.collect()

# =====================================
# 10. Execute the Training Pipeline
# =====================================

def main():
    TEST_MODE = False
    SAMPLE_SIZE = 1000

    logger.info("Starting data loading and preprocessing...")
    corpus = load_dataset(dataset_path, stop_words, test_mode=TEST_MODE, sample_size=SAMPLE_SIZE)
    if not corpus:
        logger.error("No data to train on.")
        return

    train_and_save_models(corpus, params_df, embedding_models, stop_words)

if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-03-19 19:50:39 - INFO - Loading and preprocessing additional stop words...
INFO:BERTopic_Training:Loading and preprocessing additional stop words...
2025-03-19 19:50:39 - INFO - Processed stop words count: 5181
INFO:BERTopic_Training:Processed stop words count: 5181
2025-03-19 19:50:39 - INFO - Total stop words after preprocessing: 5284
INFO:BERTopic_Training:Total stop words after preprocessing: 5284
2025-03-19 19:50:39 - INFO - Using device for embeddings: cuda
INFO:BERTopic_Training:Using device for embeddings: cuda
2025-03-19 19:50:45 - INFO - Model paraphrase-mpnet-base-v2 loaded successfully.
INFO:BERTopic_Training:Model paraphrase-

KeyboardInterrupt: 